In [ ]:
#!pip install sharepy
import sharepy 
import os
#System variables
MAIL_NUB = os.environ.get('MAIL_NUB')
PASS_MAIL_NUB = os.environ.get('PASS_MAIL_NUB')
s = sharepy.connect("nubsieteocho.sharepoint.com", MAIL_NUB, PASS_MAIL_NUB)

In [ ]:
import smtplib
import pandas as pd
from datetime import datetime, timedelta
from pandas import Timestamp

from email import encoders
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from string import printable

In [ ]:
#import data
p = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/CAB%20Trackers/Daily%20CAB%20Tracker.xlsx",
              filename='Daily CAB Tracker.xlsx')
df_compass = pd.read_excel('Daily CAB Tracker.xlsx', sheet_name='Tracker')
df_compass.dropna(axis=0, how='all', subset=['Request ID', 'Customer Name'], inplace=True)


In [ ]:
# SMTP Mail variables
today = datetime.strftime(datetime.today(), '%m-%d-%Y')
myMail = 'testNub78@outlook.com'
password = 'EasyP4ssword78'
subjectEmail = 'Coverage error report'

msg1 = """<h3>Hola, %s</h3>
        <p><span style="color: rgb(71, 85, 119);">Parece que tienes varios errores en la informaci&oacute;n registrada en tus cuentas.</span></p>
        <p><span style="color: rgb(71, 85, 119);">Asegurate de revisar el archivo adjunto para validar los campos que est&aacute;n generando conflictos.</span></p>
        <p><span style="color: rgb(71, 85, 119);">Feliz D&iacute;a.</span></p>"""


In [ ]:
def format_date_column(serie):
    format_date = []
    for index, value in serie.iteritems():
        if type(value) != datetime and type(value) != Timestamp:
            if pd.isnull(value) or value != value:
                # Fechas vacias
                value = datetime.strptime(
                    '2030-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
            else:
                # Error de formato
                value = datetime.strptime(
                    '2050-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
        else:
            # Fechas normales
            value = datetime.strptime(str(value), '%Y-%m-%d %H:%M:%S')
            format_date.append(value)

    return format_date


In [ ]:
# Advisor // Advisor Name // Date Advisor Assigned

Advisor = df_compass[['Request ID', 'RR (internal)', 'Advisor', 'Advisor Name', 'Date Advisor Assigned']].copy()
Advisor['Date Advisor Assigned'] = format_date_column(Advisor['Date Advisor Assigned']) 

for id, row in Advisor.iterrows():
    if row['RR (internal)'] == 'Yes':
        if pd.isnull(row['Advisor']) == True:
            Advisor.loc[id, 'Advisor'] = 'Empty'
        else:
            Advisor.loc[id, 'Advisor'] = None

        if pd.isnull(row['Advisor Name']) == True:
            Advisor.loc[id, 'Advisor Name'] = 'Empty Name'
        else:
            Advisor.loc[id, 'Advisor Name'] = None

        if row['Date Advisor Assigned'] == Timestamp('2030-2-2 00:00:00'):
            Advisor.loc[id, 'Date Advisor Assigned'] = 'Empty date'
        elif row['Date Advisor Assigned'] == Timestamp('2050-2-2 00:00:00'):
            Advisor.loc[id, 'Date Advisor Assigned'] = 'Format Error'
        else:
            Advisor.loc[id, 'Date Advisor Assigned'] = None    
    else:
        Advisor.loc[id, 'Advisor'] = None
        Advisor.loc[id, 'Advisor Name'] = None
        Advisor.loc[id, 'Date Advisor Assigned'] = None

Advisor.dropna(axis=0, how='all', subset=['Advisor', 'Advisor Name', 'Date Advisor Assigned'], inplace=True)
Advisor.drop(axis=1, columns=['RR (internal)'], inplace=True)

In [ ]:
# Date Readaout Done

dat_rdone = df_compass[['Request ID', 'Date Advisor Assigned', 'Date Readout Done']].copy()
dat_rdone['Date Advisor Assigned'] = format_date_column(dat_rdone['Date Advisor Assigned']) 
dat_rdone['Date Readout Done'] = format_date_column(dat_rdone['Date Readout Done']) 

for id, row in dat_rdone.iterrows():
    if row['Date Readout Done'] != Timestamp('2030-2-2 00:00:00') and row['Date Readout Done'] != Timestamp('2050-2-2 00:00:00'):
        if row['Date Advisor Assigned'] != Timestamp('2030-2-2 00:00:00') and row['Date Advisor Assigned'] != Timestamp('2050-2-2 00:00:00'):
            if row['Date Advisor Assigned'] > row['Date Readout Done']:
                dat_rdone.loc[id, 'Date Readout Done'] = 'Date Advisor Assigned greater'
            else:
                dat_rdone.loc[id, 'Date Readout Done'] = None
        else:
            dat_rdone.loc[id, 'Date Readout Done'] = None 
    elif row['Date Readout Done'] == Timestamp('2030-2-2 00:00:00') and row['Date Advisor Assigned'] != Timestamp('2030-2-2 00:00:00'):
        dat_rdone.loc[id, 'Date Readout Done'] = 'Empty Date'
    elif row['Date Readout Done'] == Timestamp('2050-2-2 00:00:00'):
        dat_rdone.loc[id, 'Date Readout Done'] = 'Format error'
    else:
        dat_rdone.loc[id, 'Date Readout Done'] = None

dat_rdone.dropna(axis=0, subset=['Date Readout Done'], inplace=True)
dat_rdone.drop(axis=1, columns=['Date Advisor Assigned'], inplace=True)

In [ ]:
# Readout delivery status

del_status = df_compass[['Request ID',
                         'Date Readout Done', 'Readout Delivery Status']].copy()

for id, row in del_status.iterrows():
    if pd.isna(row['Date Readout Done']) == False and pd.isna(row['Readout Delivery Status']) == True:
        del_status.loc[id, 'Readout Delivery Status'] = 'Empty value'
    else:
        del_status.loc[id, 'Readout Delivery Status'] = None

del_status.dropna(axis=0, subset=['Readout Delivery Status'], inplace=True)
del_status.drop(axis=1, columns=['Date Readout Done'], inplace=True)


In [ ]:
# Reason Declined // Item recommended

res_declined = df_compass[['Request ID', 'Readout Delivery Status',
                           'Reason Declined', 'Items Recommended']].copy()

for id, row in res_declined.iterrows():
    if row['Readout Delivery Status'] == 'Declined' and pd.isna(row['Reason Declined']) == True:
        res_declined.loc[id, 'Reason Declined'] = 'Empty value'
    elif (row['Readout Delivery Status'] == 'Complete' or row['Readout Delivery Status'] == 'Emailed Recommendations') and pd.isna(row['Items Recommended']) == True:
        res_declined.loc[id, 'Items Recommended'] = 'Empty value'
    else:
        res_declined.loc[id, 'Reason Declined'] = None
        res_declined.loc[id, 'Items Recommended'] = None

res_declined.dropna(axis=0, how='all', subset=[
                    'Reason Declined', 'Items Recommended'], inplace=True)
res_declined.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)


In [ ]:
# Mayor Renewal QTR

ren_qtr = df_compass[['Request ID', 'Major Renewal QTR', 'Date Presentation Created']] 
ren_qtr = ren_qtr[pd.isna(ren_qtr['Major Renewal QTR']) & pd.notna(ren_qtr['Date Presentation Created'])]
ren_qtr.fillna('Empty value', inplace=True)
ren_qtr.drop(axis=1, columns='Date Presentation Created', inplace=True)

In [ ]:
# Date presentation Created
# Note: The column date readout done do not have the correct format (it's string)

pre_created = df_compass[['Request ID', 'Date Advisor Assigned', 'Date Presentation Created']].copy()
pre_created['Date Advisor Assigned'] = format_date_column(pre_created['Date Advisor Assigned']) 
pre_created['Date Presentation Created'] = format_date_column(pre_created['Date Presentation Created']) 
pre_created = pre_created[(pre_created['Date Presentation Created'] < pre_created['Date Advisor Assigned'])]
pre_created.drop(labels=pre_created[pre_created['Date Advisor Assigned'] == '2030-02-02'].index, axis=0, inplace=True)
pre_created.drop(labels=pre_created[pre_created['Date Advisor Assigned'] == '2050-02-02'].index, axis=0, inplace=True)
pre_created['Date Presentation Created'] = 'Date Advisor Assigned is greater'
pre_created.drop(axis=1, columns=['Date Advisor Assigned'], inplace=True)

In [ ]:
# Presentation Creation Time (min)

pre_created_tim = df_compass[['Request ID', 'Date Presentation Created', 'Presentation Creation Time (min)']].copy()
pre_created_tim = pre_created_tim[(pd.notna(pre_created_tim['Date Presentation Created'])) & (pd.isna(pre_created_tim['Presentation Creation Time (min)']))]
pre_created_tim['Presentation Creation Time (min)'] = 'Empty value'
pre_created_tim.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# Readout Duration (min)

re_duration_m = df_compass[['Request ID', 'Readout Delivery Status', 'Readout Duration (min)']].copy()
re_duration_m = re_duration_m[(re_duration_m['Readout Delivery Status'] == 'Complete') | (re_duration_m['Readout Delivery Status'] == 'Emailed Recommendations')]
re_duration_m = re_duration_m[pd.isna(re_duration_m['Readout Duration (min)'])]
re_duration_m['Readout Duration (min)'] = 'Empty value'
re_duration_m.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# Date Invite Sent

dat_invite_s = df_compass[['Request ID', 'Date Advisor Assigned', 'Date Invite Sent']].copy()
dat_invite_s['Date Advisor Assigned'] = format_date_column(dat_invite_s['Date Advisor Assigned']) 
dat_invite_s['Date Invite Sent'] = format_date_column(dat_invite_s['Date Invite Sent'])
dat_invite_s = dat_invite_s[dat_invite_s['Date Advisor Assigned'] > dat_invite_s['Date Invite Sent']]
dat_invite_s.drop(labels=dat_invite_s[dat_invite_s['Date Advisor Assigned'] == '2030-02-02'].index, axis=0, inplace=True)
dat_invite_s.drop(labels=dat_invite_s[dat_invite_s['Date Advisor Assigned'] == '2050-02-02'].index, axis=0, inplace=True)
dat_invite_s['Date Invite Sent'] = 'Date Advisor Assigned greater'
dat_invite_s.drop(axis=1, columns=['Date Advisor Assigned'], inplace=True)

In [ ]:
# Date Summary Sent
# En pausa dado que la columna readout done tiene formato string

In [ ]:
# IB value (in M) // Cover the Uncovered Presented

ibvalue_and_cover = df_compass[['Request ID', 'Date Presentation Created', 'IB value (in M)', 'Cover the Uncovered Presented']].copy()
ibvalue_and_cover = ibvalue_and_cover[pd.notna(ibvalue_and_cover['Date Presentation Created'])]
ibvalue_and_cover = ibvalue_and_cover[(pd.isna(ibvalue_and_cover['IB value (in M)'])) | (pd.isna(ibvalue_and_cover['Cover the Uncovered Presented']))]
ibvalue_and_cover.fillna('Empty value', inplace=True)
ibvalue_and_cover['Cover the Uncovered Presented'] = ibvalue_and_cover['Cover the Uncovered Presented'].apply(
    lambda x: None if x != 'Empty value' else x)
ibvalue_and_cover['IB value (in M)'] = ibvalue_and_cover['IB value (in M)'].apply(
    lambda x: None if x != 'Empty value' else x)
ibvalue_and_cover.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# AT Interested (CTU)

at_interested = df_compass[['Request ID', 'Readout Delivery Status', 'AT Interested (CTU)']].copy()
at_interested = at_interested[(at_interested['Readout Delivery Status'] == 'Complete') &
                pd.isnull(at_interested['AT Interested (CTU)'])]
at_interested['AT Interested (CTU)'] = 'Empty value'
at_interested.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# Merge

final_report = Advisor.merge(dat_rdone, how='outer', left_on='Request ID', right_on='Request ID').merge(
    del_status, how='outer', left_on='Request ID', right_on='Request ID').merge(
    res_declined, how='outer', left_on='Request ID', right_on='Request ID').merge(
    ren_qtr, how='outer', left_on='Request ID', right_on='Request ID').merge(
    pre_created, how='outer', left_on='Request ID', right_on='Request ID').merge(
    pre_created_tim, how='outer', left_on='Request ID', right_on='Request ID').merge(
    re_duration_m, how='outer', left_on='Request ID', right_on='Request ID').merge(
    dat_invite_s, how='outer', left_on='Request ID', right_on='Request ID').merge(
    ibvalue_and_cover, how='outer', left_on='Request ID', right_on='Request ID').merge(
    at_interested, how='outer', left_on='Request ID', right_on='Request ID')

In [ ]:
final_report[final_report['Request ID'].duplicated()]